In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from mrtool import MRData, LinearCovModel
from gkmodel import OverallModel, StudyModel, TwoStageModel, result_to_df

In [ ]:
indicator = 'smoking'
df = pd.read_csv(f'../data/{indicator}.as.csv')

In [ ]:
df.head(3)

In [ ]:
data_stage1 = MRData()
data_stage1.load_df(
    df,
    col_obs=f'{indicator}_logit',
    col_obs_se=f'{indicator}_logit_se',
    col_covs=['sdi', 'year_id'],
    col_study_id='ls_id'
)

### Using Overall Model and StudyModel separately

In [ ]:
cov_models = [
    LinearCovModel('intercept'),
    LinearCovModel('sdi',
                   use_spline=True,
                   spline_knots=np.linspace(0.0, 1.0, 5),
                   spline_l_linear=True,
                   spline_r_linear=True)
]

In [ ]:
data_stage1

In [ ]:
model_stage1 = OverallModel(data_stage1, cov_models)

In [ ]:
model_stage1.fit_model()

In [ ]:
model_stage1.soln

In [ ]:
model_stage1.data._sort_by_data_id()
df_stage1 = result_to_df(model_stage1, model_stage1.data, prediction='pred_stage1', residual='resi_stage1')
df_stage1 = pd.merge(df, df_stage1[['pred_stage1', 'resi_stage1']], left_index=True, right_index=True)

In [ ]:
df_stage1.head()

In [ ]:
data_stage2 = MRData()
data_stage2.load_df(
    df_stage1,
    col_obs='resi_stage1',
    col_obs_se=f'{indicator}_logit_se',
    col_covs=['year_id'],
    col_study_id='ls_id'
)

In [ ]:
model_stage2 = StudyModel(data_stage2, ['intercept', 'year_id'])

In [ ]:
model_stage2.fit_model()

### Using TwoStage Model

In [ ]:
cov_models1 = [
    LinearCovModel('intercept'),
    LinearCovModel('sdi',
                   use_spline=True,
                   spline_knots=np.linspace(0.0, 1.0, 5),
                   spline_l_linear=True,
                   spline_r_linear=True)
]
cov_models2 = [LinearCovModel('intercept'), LinearCovModel('year_id')]

In [ ]:
tsmodel = TwoStageModel(data_stage1, cov_models1, cov_models2)

In [ ]:
tsmodel.fit_model()

In [ ]:
tsmodel.write_stage1_soln()

In [ ]:
tsmodel.write_stage2_soln()

In [ ]:
result_to_df(tsmodel, tsmodel.data1)

### Compare Fit

In [ ]:
print(model_stage1.soln - tsmodel.model1.soln)

In [ ]:
for ls_id, soln in model_stage2.soln.items():
    assert np.linalg.norm(soln - tsmodel.model2.soln[ls_id]) / np.linalg.norm(soln) < 1e-5

### Compare prediction

- predictions from OverallModel + StudyModel

In [ ]:
model_stage2.data._sort_by_data_id()
df_stage2 = result_to_df(model_stage2, model_stage2.data, prediction='pred_stage2', residual='resi_stage2')
df_stage2 = pd.merge(df_stage1, df_stage2[['pred_stage2', 'resi_stage2']], left_index=True, right_index=True)

In [ ]:
df_stage2.head()

In [ ]:
prediction = df_stage2['pred_stage1'].values + df_stage2['pred_stage2'].values

- prediction from TwoStageModel

In [ ]:
prediction_ts = tsmodel.predict()

In [ ]:
np.linalg.norm(prediction_ts - prediction)

- predict with quantile

In [ ]:
tsmodel.predict(random_slope=dict(year_id=0.15))

### More plotting

In [ ]:
ls_id = 1021
index = df_stage2.ls_id == ls_id
obs = df_stage2[f'{indicator}_logit'].values[index]
year_id = df_stage2['year_id'].values[index]

prediction = df_stage2['pred_stage1'].values + df_stage2['pred_stage2'].values
print(prediction)
prediction = prediction[index]

plt.scatter(year_id, obs, label='observation')
plt.scatter(year_id, prediction, label='prediction')
plt.legend()
plt.title(f"loc_sex_id: {ls_id}")

In [ ]:
soln = np.vstack(list(model_stage2.soln.values()))

In [ ]:
plt.plot(np.sort(soln[:, 0]), np.linspace(0.0, 1.0, soln.shape[0]))
plt.title("random intercept cumulative density")

In [ ]:
plt.plot(np.sort(soln[:, 1]), np.linspace(0.0, 1.0, soln.shape[0]))
plt.title("random slope cumulative density")